In [ ]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
import itertools
from IPython.display import display

plt.rcParams['figure.figsize'] = [14, 8]

In [17]:
base_path = "results/"
folders= ["share", "no_share"]
tests= [
    "multiplexer", 
    "parity", 
    "regression", 
    "multiplexer_n1000", 
    "parity_n1000",
    "regression_n1000",
    "multiplexer_v2",
    "parity_v2",
    "regression_v2", 
    "old_regression", 
    "older_regression",
    "gym",
]
runs = ["1", "2", "3", "4", "5"]

col_names = []

dfs = {}
for t in tests: 
    dfs[t] = {}
    for f in folders:
        dfs[t][f] = []
        for r in runs:
            file_name = base_path + f + "/diversity_" + t + "_" + r  + ".csv"
#             print(file_name)
            df = pd.read_csv(file_name, sep=';')
            dfs[t][f].append(df)
            col_names = df.columns[1:]
            
dfs[t][f][0].head(2)

,gen,fitness,n_species,p_isomorphisms,unique_fitness,entropy,total_gen_markers,active_gen_markers,inactive_gen_markers,avg_delta,std_delta
0,0,9.259512,421,246,79.0,5.832377,26553.0,3010.0,24734.0,0.736030,0.026661
1,1,5.684596,189,259,83.0,5.946469,27111.0,3018.0,25321.0,0.726626,0.034749


In [18]:
def normalize_column(df, c_name):
    c = df[c_name]
    df[c_name] = (c-c.min())/(c.max()-c.min())

In [19]:
def plot_graphs(dfs, test_name, gen_range, cols, method):
    for k, v in dfs[test_name].items():
        flag = False
        for m in method:
            if k == m.description and m.value == False:
                flag = True
                break
        if flag: 
            continue
        for column in cols:
            full_data = [d[column] for d in v]
            
            data = []
            for d in itertools.zip_longest(*full_data, fillvalue = 0):
                data.append(d)
                
            start = min(gen_range[0], len(data))
            finish = min(gen_range[1], len(data))
            steps = [i for i in range(start, finish)]
            
            data = data[start:finish]
            avg_arr = [sum(d)/len(d) for d in data]
            min_arr = [min(d) for d in data]
            max_arr = [max(d) for d in data]

            plt.plot(steps, avg_arr, lw =1.3, label=f"{k} {column}")
            plt.fill_between(steps, min_arr,max_arr, alpha = 0.15)

    plt.legend(loc="upper left")
    plt.show()

In [24]:
test = widgets.ToggleButtons(
    options=tests,
    description='Test:',
    disabled=False,
    button_style='',
    indent=True
)

share_method = widgets.ToggleButton(
    value=True,
    description='share',
    disabled=False,
    button_style='', 
    tooltip='Description',
)
no_share_method = widgets.ToggleButton(
    value=True,
    description='no_share',
    disabled=False,
    button_style='',
    tooltip='Description',
)
method = [share_method, no_share_method]


gen_range = widgets.IntRangeSlider(
    value=[10, 1000],
    min=0,
    max=1000,
    step=10,
    description='Generations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

parameters = []
for col in col_names:
    cb = widgets.Checkbox(
        value=False,
        description=col,
        disabled=False,
        indent=False
    )
    parameters.append(cb)
    
button = widgets.Button(description="Render Graph")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        cols = [p.description for p in parameters if p.value]
        if len(cols) > 0:
            plot_graphs(dfs, test.value, gen_range.value, cols, method)

button.on_click(on_button_clicked)

In [25]:
display(test)
display(widgets.HBox([widgets.Label(value="Method: "), *method]))
display(gen_range)

half_len = len(parameters)//2
input_widgets_1 = widgets.HBox(parameters[:half_len])
display(input_widgets_1)
input_widgets_2 = widgets.HBox(parameters[half_len:])
display(input_widgets_2)

display(button, output)

ToggleButtons(description='Test:', options=('multiplexer', 'parity', 'regression', 'multiplexer_n1000', 'parit…

IntRangeSlider(value=(10, 1000), continuous_update=False, description='Generations:', max=1000, step=10)

Button(description='Render Graph', style=ButtonStyle())

Output()